In [1]:
import sqlite3
from contextlib import contextmanager
import uuid
import numpy as np
from pydub import AudioSegment
from scipy.ndimage import maximum_filter
from scipy.signal import spectrogram
from tinytag import TinyTag
from multiprocessing import Pool, Lock, current_process
import os
from collections import defaultdict

In [2]:
SAMPLE_RATE = 44100
PEAK_BOX_SIZE = 30
POINT_EFFICIENCY = 0.8
TARGET_START = 0.05
TARGET_T = 1.8
TARGET_F = 4000
FFT_WINDOW_SIZE = 0.2
DB_PATH = "hash.db"
NUM_WORKERS = 24
KNOWN_EXTENSIONS = ["mp3", "wav", "flac", "m4a"]

## Настройка базы данных

In [38]:
@contextmanager
def get_cursor():
    """Get a connection/cursor to the database.

    :returns: Tuple of connection and cursor.
    """
    conn = sqlite3.connect(DB_PATH, timeout=30)
    try:
        yield conn, conn.cursor()
    finally:
        conn.close()

with get_cursor() as (conn, c):
    c.execute("CREATE TABLE IF NOT EXISTS hash (hash int, offset real, song_id text)")
    c.execute("CREATE TABLE IF NOT EXISTS song_info (artist text, album text, title text, song_id text)")
    # dramatically speed up recognition
    c.execute("CREATE INDEX IF NOT EXISTS idx_hash ON hash (hash)")
    # faster write mode that enables greater concurrency
    # https://sqlite.org/wal.html
    c.execute("PRAGMA journal_mode=WAL")
    # reduce load at a checkpoint and reduce chance of a timeout
    c.execute("PRAGMA wal_autocheckpoint=300")


In [47]:
SAMPLE_RATE = 44100
PEAK_BOX_SIZE = 30
POINT_EFFICIENCY = 0.8
TARGET_START = 0.05
TARGET_T = 1.8
TARGET_F = 4000
FFT_WINDOW_SIZE = 0.2
DB_PATH = "hash.db"
NUM_WORKERS = 24

@contextmanager
def get_cursor():
    """Get a connection/cursor to the database.

    :returns: Tuple of connection and cursor.
    """
    conn = sqlite3.connect(DB_PATH, timeout=30)
    try:
        yield conn, conn.cursor()
    finally:
        conn.close()

with get_cursor() as (conn, c):
    c.execute("CREATE TABLE IF NOT EXISTS hash (hash int, offset real, song_id text)")
    c.execute("CREATE INDEX IF NOT EXISTS idx_hash ON hash (hash)")
    c.execute("PRAGMA journal_mode=WAL")
    c.execute("PRAGMA wal_autocheckpoint=300")

KNOWN_EXTENSIONS = ["mp3", "wav", "flac", "m4a"]

def my_spectrogram(audio):
    """Helper function that performs a spectrogram with the values in settings."""
    nperseg = int(SAMPLE_RATE * FFT_WINDOW_SIZE)
    return spectrogram(audio, SAMPLE_RATE, nperseg=nperseg)

def file_to_spectrogram(filename):
    """Calculates the spectrogram of a file.

    Converts a file to mono and resamples to :data:`~abracadabra.settings.SAMPLE_RATE` before
    calculating. Uses :data:`~abracadabra.settings.FFT_WINDOW_SIZE` for the window size.

    :param filename: Path to the file to spectrogram.
    :returns: * f - list of frequencies
              * t - list of times
              * Sxx - Power value for each time/frequency pair
    """
    a = AudioSegment.from_file(filename).set_channels(1).set_frame_rate(SAMPLE_RATE)
    audio = np.frombuffer(a.raw_data, np.int16)
    return my_spectrogram(audio)

def find_peaks(Sxx):
    """Finds peaks in a spectrogram.

    Uses :data:`~abracadabra.settings.PEAK_BOX_SIZE` as the size of the region around each
    peak. Calculates number of peaks to return based on how many peaks could theoretically
    fit in the spectrogram and the :data:`~abracadabra.settings.POINT_EFFICIENCY`.

    Inspired by
    `photutils
    <https://photutils.readthedocs.io/en/stable/_modules/photutils/detection/core.html#find_peaks>`_.

    :param Sxx: The spectrogram.
    :returns: A list of peaks in the spectrogram.
    """
    data_max = maximum_filter(Sxx, size=PEAK_BOX_SIZE, mode='constant', cval=0.0)
    peak_goodmask = (Sxx == data_max)  # good pixels are True
    y_peaks, x_peaks = peak_goodmask.nonzero()
    peak_values = Sxx[y_peaks, x_peaks]
    i = peak_values.argsort()[::-1]
    # get co-ordinates into arr
    j = [(y_peaks[idx], x_peaks[idx]) for idx in i]
    total = Sxx.shape[0] * Sxx.shape[1]
    # in a square with a perfectly spaced grid, we could fit area / PEAK_BOX_SIZE^2 points
    # use point efficiency to reduce this, since it won't be perfectly spaced
    # accuracy vs speed tradeoff
    peak_target = int((total / (PEAK_BOX_SIZE**2)) * POINT_EFFICIENCY)
    return j[:peak_target]

def idxs_to_tf_pairs(idxs, t, f):
    """Helper function to convert time/frequency indices into values."""
    return np.array([(f[i[0]], t[i[1]]) for i in idxs])

def hash_point_pair(p1, p2):
    """Helper function to generate a hash from two time/frequency points."""
    return hash((p1[0], p2[0], p2[1]-p2[1]))

def target_zone(anchor, points, width, height, t):
    """Generates a target zone as described in `the Shazam paper
    <https://www.ee.columbia.edu/~dpwe/papers/Wang03-shazam.pdf>`_.

    Given an anchor point, yields all points within a box that starts `t` seconds after the point,
    and has width `width` and height `height`.

    :param anchor: The anchor point
    :param points: The list of points to search
    :param width: The width of the target zone
    :param height: The height of the target zone
    :param t: How many seconds after the anchor point the target zone should start
    :returns: Yields all points within the target zone.
    """
    x_min = anchor[1] + t
    x_max = x_min + width
    y_min = anchor[0] - (height*0.5)
    y_max = y_min + height
    for point in points:
        if point[0] < y_min or point[0] > y_max:
            continue
        if point[1] < x_min or point[1] > x_max:
            continue
        yield point

def hash_points(points, filename):
    """Generates all hashes for a list of peaks.

    Iterates through the peaks, generating a hash for each peak within that peak's target zone.

    :param points: The list of peaks.
    :param filename: The filename of the song, used for generating song_id.
    :returns: A list of tuples of the form (hash, time offset, song_id).
    """
    hashes = []
    song_name = os.path.splitext(os.path.basename(filename))[0]  # Get only the file name without extension
    for anchor in points:
        for target in target_zone(
            anchor, points, TARGET_T, TARGET_F, TARGET_START
        ):
            hashes.append((
                # hash
                hash_point_pair(anchor, target),
                # time offset
                anchor[1],
                # filename
                str(song_name)  # Use only the file name without extension
            ))
    return hashes



def fingerprint_file(filename):
    """Generate hashes for a file.

    Given a file, runs it through the fingerprint process to produce a list of hashes from it.

    :param filename: The path to the file.
    :returns: The output of :func:`hash_points`.
    """
    f, t, Sxx = file_to_spectrogram(filename)
    peaks = find_peaks(Sxx)
    peaks = idxs_to_tf_pairs(peaks, t, f)
    return hash_points(peaks, filename)

def store_song(hashes):
    """Register a song in the database.

    :param hashes: A list of tuples of the form (hash, time offset, song_id) as returned by
        :func:`~abracadabra.fingerprint.fingerprint_file`.
    """
    if len(hashes) < 1:
        # TODO: After experiments have run, change this to raise error
        # Probably should re-run the peaks finding with higher efficiency
        # or maybe widen the target zone
        return
    with get_cursor() as (conn, c):
        c.executemany("INSERT INTO hash VALUES (?, ?, ?)", hashes)
        conn.commit()

def song_in_db(filename):
    """Check whether a path has already been registered.

    :param filename: The path to check.
    :returns: Whether the path exists in the database yet.
    :rtype: bool
    """
    with get_cursor() as (conn, c):
        c.execute("SELECT * FROM hash WHERE song_id=?", (filename,))
        return c.fetchone() is not None

def register_song(filename):
    """Register a single song.

    Checks if the song is already registered based on path provided and ignores
    those that are already registered.

    :param filename: Path to the file to register"""
    print("Registering song:", filename)
    if song_in_db(filename):
        print("Song already registered, skipping:", filename)
        return
    print("Song not registered yet, generating hashes...")
    hashes = fingerprint_file(filename)
    print("Hashes generated for song:", filename)
    print("Writing to database...")
    store_song(hashes)
    print("Finished registering song:", filename)

def register_directory(path):
    """Recursively register songs in a directory.

    Registers songs in a directory sequentially.

    :param path: Path of directory to register
    """
    print("Start registering directory:", path)
    to_register = []
    for root, _, files in os.walk(path):
        for f in files:
            if f.split('.')[-1] not in KNOWN_EXTENSIONS:
                continue
            file_path = os.path.join(root, f)
            to_register.append(file_path)
    print("Files to register:", len(to_register))
    for file_path in to_register:
        register_song(file_path)
    print("Finished registering directory:", path)

register_directory('songs')


Start registering directory: songs
Files to register: 11
Registering song: songs\7 Seconds - Youssou N'Dour, Neneh Cherry.mp3
Song not registered yet, generating hashes...
Hashes generated for song: songs\7 Seconds - Youssou N'Dour, Neneh Cherry.mp3
Writing to database...
Finished registering song: songs\7 Seconds - Youssou N'Dour, Neneh Cherry.mp3
Registering song: songs\a-ha - Take On Me.mp3
Song not registered yet, generating hashes...
Hashes generated for song: songs\a-ha - Take On Me.mp3
Writing to database...
Finished registering song: songs\a-ha - Take On Me.mp3
Registering song: songs\ABBA - Money, Money, Money.mp3
Song not registered yet, generating hashes...
Hashes generated for song: songs\ABBA - Money, Money, Money.mp3
Writing to database...
Finished registering song: songs\ABBA - Money, Money, Money.mp3
Registering song: songs\ABBA - The Winner Takes It All.mp3
Song not registered yet, generating hashes...
Hashes generated for song: songs\ABBA - The Winner Takes It All.mp3

In [48]:
def score_match(offsets):
    """Score a matched song.

    Calculates a histogram of the deltas between the time offsets of the hashes from the
    recorded sample and the time offsets of the hashes matched in the database for a song.
    The function then returns the size of the largest bin in this histogram as a score.

    :param offsets: List of offset pairs for matching hashes
    :returns: The highest peak in a histogram of time deltas
    :rtype: int
    """
    # Use bins spaced 0.5 seconds apart
    binwidth = 0.5
    tks = list(map(lambda x: x[0] - x[1], offsets))
    hist, _ = np.histogram(tks,
                           bins=np.arange(int(min(tks)),
                                          int(max(tks)) + binwidth + 1,
                                          binwidth))
    return np.max(hist)

def best_match(matches):
    """For a dictionary of song_id: offsets, returns the best song_id.

    Scores each song in the matches dictionary and then returns the song_id with the best score.

    :param matches: Dictionary of song_id to list of offset pairs (db_offset, sample_offset)
       as returned by :func:`~abracadabra.Storage.storage.get_matches`.
    :returns: song_id with the best score.
    :rtype: str
    """
    matched_song = None
    best_score = 0
    for song_id, offsets in matches.items():
        if len(offsets) < best_score:
            # can't be best score, avoid expensive histogram
            continue
        score = score_match(offsets)
        if score > best_score:
            best_score = score
            matched_song = song_id
    return matched_song

def get_matches(hashes, threshold=5):
    """Get matching songs for a set of hashes.

    :param hashes: A list of hashes as returned by
        :func:`~abracadabra.fingerprint.fingerprint_file`.
    :param threshold: Return songs that have more than ``threshold`` matches.
    :returns: A dictionary mapping ``song_id`` to a list of time offset tuples. The tuples are of
        the form (result offset, original hash offset).
    :rtype: dict(str: list(tuple(float, float)))
    """
    h_dict = {}
    for h, t, _ in hashes:
        h_dict[h] = t
    in_values = f"({','.join([str(h[0]) for h in hashes])})"
    with get_cursor() as (conn, c):
        c.execute(f"SELECT hash, offset, song_id FROM hash WHERE hash IN {in_values}")
        results = c.fetchall()
    result_dict = defaultdict(list)
    for r in results:
        result_dict[r[2]].append((r[1], h_dict[r[0]]))
    return result_dict

def recognise_song(filename):
    """Recognises a pre-recorded sample.

    Recognises the sample stored at the path ``filename``. The sample can be in any of the
    formats in :data:`recognise.KNOWN_FORMATS`.

    :param filename: Path of file to be recognised.
    :returns: :func:`~abracadabra.recognise.get_song_info` result for matched song or None.
    :rtype: tuple(str, str, str)
    """
    hashes = fingerprint_file(filename)
    matches = get_matches(hashes)
    matched_song = best_match(matches)
    return matched_song

print(recognise_song('test_data/Timbaland_-_Apologize_47972715.mp3'))

Apologize


In [41]:
def score_match(offsets):
    """Score a matched song.

    Calculates a histogram of the deltas between the time offsets of the hashes from the
    recorded sample and the time offsets of the hashes matched in the database for a song.
    The function then returns the size of the largest bin in this histogram as a score.

    :param offsets: List of offset pairs for matching hashes
    :returns: The highest peak in a histogram of time deltas
    :rtype: int
    """
    # Use bins spaced 0.5 seconds apart
    binwidth = 0.5
    tks = list(map(lambda x: x[0] - x[1], offsets))
    hist, _ = np.histogram(tks,
                           bins=np.arange(int(min(tks)),
                                          int(max(tks)) + binwidth + 1,
                                          binwidth))
    return np.max(hist)

def best_matches(matches, num_matches=5):
    """For a dictionary of song_id: offsets, returns the best song_id along with similarity score.

    Scores each song in the matches dictionary and then returns the top num_matches songs along with their scores.

    :param matches: Dictionary of song_id to list of offset pairs (db_offset, sample_offset)
       as returned by :func:`~abracadabra.Storage.storage.get_matches`.
    :param num_matches: Number of best matches to return.
    :returns: List of tuples containing (song_id, similarity_score).
    :rtype: list(tuple(str, float))
    """
    scored_matches = []
    for song_id, offsets in matches.items():
        score = score_match(offsets)
        scored_matches.append((song_id, score))
    # Sort matches based on score in descending order
    scored_matches.sort(key=lambda x: x[1], reverse=True)
    # Return top num_matches matches
    return scored_matches[:num_matches]


def get_matches(hashes, threshold=5):
    """Get matching songs for a set of hashes.

    :param hashes: A list of hashes as returned by
        :func:`~abracadabra.fingerprint.fingerprint_file`.
    :param threshold: Return songs that have more than ``threshold`` matches.
    :returns: A dictionary mapping ``song_id`` to a list of time offset tuples. The tuples are of
        the form (result offset, original hash offset).
    :rtype: dict(str: list(tuple(float, float)))
    """
    h_dict = {}
    for h, t, _ in hashes:
        h_dict[h] = t
    in_values = f"({','.join([str(h[0]) for h in hashes])})"
    with get_cursor() as (conn, c):
        c.execute(f"SELECT hash, offset, song_id FROM hash WHERE hash IN {in_values}")
        results = c.fetchall()
    result_dict = defaultdict(list)
    for r in results:
        result_dict[r[2]].append((r[1], h_dict[r[0]]))
    return result_dict

def recognise_song(filename, num_matches=5):
    """Recognises a pre-recorded sample.

    Recognises the sample stored at the path ``filename``. The sample can be in any of the
    formats in :data:`recognise.KNOWN_FORMATS`.

    :param filename: Path of file to be recognised.
    :param num_matches: Number of best matches to return.
    :returns: List of tuples containing (song_id, similarity_score).
    :rtype: list(tuple(str, float))
    """
    hashes = fingerprint_file(filename)
    matches = get_matches(hashes)
    best_matches_result = best_matches(matches, num_matches)
    return best_matches_result



print(recognise_song('test_data/Timbaland_-_Apologize_47972715.mp3'))

[('Apologize', 837), ('B', 7), ('Bag Raiders -- Shooting stars', 5), ('Bag Raiders - Shooting stars', 5), ('Bad Romance', 5)]


In [51]:
import os
import csv

# Функция для чтения ожидаемых названий файлов в папке "test"
def read_expected_names(folder):
    expected_names = []
    for filename in os.listdir(folder):
        if any(filename.endswith(ext) for ext in KNOWN_EXTENSIONS):
            name = os.path.splitext(os.path.basename(filename))[0]  # Get only the file name without extension
            name = name.split("_")[0]  # Получаем название без номера фрагмента и расширения
            expected_names.append((filename, name))

    return expected_names

# Функция для оценки правильности распознавания
def evaluate_recognition(folder):
    # Чтение ожидаемых названий файлов
    expected_names = read_expected_names(folder)

    # Запуск программы распознавания для каждого файла и сравнение результатов
    results = []
    correct_count = 0
    total_count = 0
    for filename, expected_name in expected_names:
        # Получение названия композиции из файла
        recognized_name = recognise_song(os.path.join(folder, filename))

        # Добавление результата в список
        results.append((expected_name, recognized_name))

        # Сравнение результатов
        if recognized_name == expected_name:
            correct_count += 1
        total_count += 1
    
    # Расчет точности распознавания
    accuracy = correct_count / total_count if total_count > 0 else 0

    return results, accuracy

# Оценка правильности распознавания для папки "test"
folder = "test"
results, accuracy = evaluate_recognition(folder)

# Сохранение результатов в CSV-файл с разделителем ";"
with open("recognition_results.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    writer.writerow(["True song name", "Recognized song name"])
    writer.writerows(results)

# Вывод точности распознавания
print("Точность распознавания:", accuracy)


[("7 Seconds - Youssou N'Dour, Neneh Cherry.mp3", "7 Seconds - Youssou N'Dour, Neneh Cherry"), ('a-ha - Take On Me.mp3', 'a-ha - Take On Me'), ('ABBA - Money, Money, Money.mp3', 'ABBA - Money, Money, Money'), ('ABBA - The Winner Takes It All.mp3', 'ABBA - The Winner Takes It All'), ('Animals - Martin Garrix.mp3', 'Animals - Martin Garrix'), ('t.a.T.u. - All The Things She Said.mp3', 't.a.T.u. - All The Things She Said')]
Точность распознавания: 1.0


In [52]:
import os
import random
from pydub import AudioSegment

# Путь к папке с исходными аудиофайлами
input_folder = "test/full"
# Путь к папке для сохранения фрагментов
output_folder = "test/snippet"
# Количество случайных фрагментов, которые нужно извлечь из каждого файла
num_snippets_per_file = 3
# Длительность фрагмента в миллисекундах (10 секунд)
snippet_duration_ms = 10 * 1000

# Создаем папку для сохранения фрагментов, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Функция для извлечения случайного фрагмента из аудиофайла
def extract_random_snippet(file_path):
    audio = AudioSegment.from_file(file_path)
    # Выбираем случайную точку начала фрагмента
    start_time = random.randint(0, len(audio) - snippet_duration_ms)
    # Извлекаем фрагмент заданной длительности
    snippet = audio[start_time:start_time + snippet_duration_ms]
    return snippet

# Обход всех аудиофайлов в папке "test/full"
for filename in os.listdir(input_folder):
    if any(filename.endswith(ext) for ext in KNOWN_EXTENSIONS):
        # Полный путь к текущему аудиофайлу
        file_path = os.path.join(input_folder, filename)
        # Извлекаем несколько случайных фрагментов из текущего аудиофайла
        for i in range(num_snippets_per_file):
            # Извлекаем случайный фрагмент
            snippet = extract_random_snippet(file_path)
            # Генерируем имя для нового файла
            new_filename = f"{os.path.splitext(filename)[0]}_snippet_{i + 1}.mp3"
            # Полный путь к новому файлу
            output_path = os.path.join(output_folder, new_filename)
            # Сохраняем фрагмент в новый файл
            snippet.export(output_path, format="mp3")

print("Случайные фрагменты успешно сохранены в формате MP3.")


Случайные фрагменты успешно сохранены в формате MP3.


## Распознавание по напеванию

In [76]:
import sqlite3
import numpy as np
import os
from collections import defaultdict
from pydub import AudioSegment
from pydub.playback import play
import librosa
import uuid
import sounddevice as sd
import soundfile as sf
import time

DB_PATH = "hash1.db"
KNOWN_EXTENSIONS = ["mp3", "wav", "flac", "m4a"]
NUM_WORKERS = 24
TARGET_T = 1.8
TARGET_F = 4000

def get_cursor():
    """Get a connection/cursor to the database."""
    conn = sqlite3.connect(DB_PATH, timeout=30)
    c = conn.cursor()
    return conn, c

def close_cursor(conn):
    """Close the database connection."""
    conn.close()

def create_tables():
    """Создать необходимые таблицы в базе данных."""
    conn, c = get_cursor()
    c.execute("CREATE TABLE IF NOT EXISTS hash (hash int, offset real, filename text)")
    c.execute("CREATE INDEX IF NOT EXISTS idx_hash ON hash (hash)")
    conn.commit()
    close_cursor(conn)


def my_chromagram(audio, sr):
    """Calculate the chromagram of an audio signal."""
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    return chroma

def file_to_chromagram(filename):
    """Convert an audio file to a chromagram."""
    audio, sr = librosa.load(filename, sr=None)
    return my_chromagram(audio, sr)

def hash_chromagram(chroma, filename):
    """Generate hashes for a chromagram."""
    hashes = []
    for i, chroma_frame in enumerate(chroma):
        for j, val in enumerate(chroma_frame):
            hashes.append((
                hash((i, j, val)),
                i,
                filename
            ))
    return hashes

def fingerprint_file(filename, start_time, end_time):
    """Generate hashes for an audio file."""
    audio, sr = librosa.load(filename, sr=None, offset=start_time, duration=end_time-start_time)
    chroma = my_chromagram(audio, sr)
    return hash_chromagram(chroma, filename)

def store_song(hashes, song_id):
    """Register a song in the database."""
    if len(hashes) < 1:
        return
    conn, c = get_cursor()
    c.executemany("INSERT INTO hash VALUES (?, ?, ?)", hashes)
    conn.commit()
    close_cursor(conn)

def song_in_db(song_id):
    """Check if a song is already registered in the database."""
    conn, c = get_cursor()
    c.execute("SELECT * FROM hash WHERE song_id=?", (song_id,))
    result = c.fetchone() is not None
    close_cursor(conn)
    return result

def register_song(filename):
    """Register a single song."""
    print("Registering song:", filename)
    song_id = str(uuid.uuid5(uuid.NAMESPACE_OID, filename).int)
    if song_in_db(song_id):
        print("Song already registered, skipping:", filename)
        return
    print("Song not registered yet, generating hashes...")
    hashes = fingerprint_file(filename)
    print("Hashes generated for song:", filename)
    print("Writing to database...")
    store_song(hashes, song_id)
    print("Finished registering song:", filename)


def register_directory(path):
    """Recursively register songs in a directory."""
    print("Start registering directory:", path)
    to_register = []
    for root, _, files in os.walk(path):
        for f in files:
            if f.split('.')[-1] not in KNOWN_EXTENSIONS:
                continue
            file_path = os.path.join(root, f)
            to_register.append(file_path)
    print("Files to register:", len(to_register))
    for file_path in to_register:
        register_song(file_path)
    print("Finished registering directory:", path)

def score_match(offsets):
    """Score a matched song."""
    binwidth = 0.5
    tks = list(map(lambda x: x[0] - x[1], offsets))
    hist, _ = np.histogram(tks, bins=np.arange(int(min(tks)), int(max(tks)) + binwidth + 1, binwidth))
    return np.max(hist)

def best_match(matches):
    """Find the best matching song."""
    matched_song = None
    best_score = 0
    for song_id, offsets in matches.items():
        if len(offsets) < best_score:
            continue
        score = score_match(offsets)
        if score > best_score:
            best_score = score
            matched_song = song_id
    return matched_song

def get_matches(hashes, threshold=5):
    """Get matching songs for a set of hashes."""
    h_dict = {}
    for h, t, _ in hashes:
        h_dict[h] = t
    in_values = f"({','.join([str(h[0]) for h in hashes])})"
    conn, c = get_cursor()
    c.execute(f"SELECT hash, offset, song_id FROM hash WHERE hash IN {in_values}")
    results = c.fetchall()
    result_dict = defaultdict(list)
    for r in results:
        if r[1] in h_dict.values():
            result_dict[r[2]].append((r[1], h_dict[r[0]]))
    close_cursor(conn)
    return result_dict


def recognise_song(filename):
    """Recognize a pre-recorded sample."""
    duration = librosa.get_duration(filename=filename)
    hashes = fingerprint_file(filename, start_time=0, end_time=duration)
    matches = get_matches(hashes)
    matched_song = best_match(matches)
    return matched_song


def record_audio(duration, filename):
    """Record audio from the microphone."""
    print("Recording...")
    audio = sd.rec(int(duration * 44100), samplerate=44100, channels=2, dtype='int16')
    sd.wait()
    sf.write(filename, audio, samplerate=44100)
    print("Recording complete!")

def hum_recognition():
    """Recognize a song based on humming."""
    print("Please hum or whistle the tune of the song you want to recognize...")
    record_audio(15, "humming.wav")
    print("Recognizing...")
    matched_song = recognise_song("humming.wav")
    if matched_song:
        print("The recognized song filename is:", matched_song)
    else:
        print("Sorry, the song could not be recognized.")

# Инициализация таблиц в базе данных перед началом работы
#create_tables()

# Регистрация директории с песнями
#register_directory('songs')

# Распознавание песни по напеванию
hum_recognition()


Please hum or whistle the tune of the song you want to recognize...
Recording...
Recording complete!
Recognizing...


C:\Users\ccoo0\AppData\Local\Temp\ipykernel_9512\2803330718.py:152: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration = librosa.get_duration(filename=filename)


The recognized song filename is: songs\Bad Romance.mp3
